In [457]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [458]:
feature_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety']
class_name = "class"

car_df = pd.read_csv('car.data', names=np.concatenate([feature_names, [class_name]]))

In [459]:
car_df.isna().sum()

,0
buying,0
maint,0
doors,0
persons,0
lug_boot,0
safety,0
class,0


In [460]:
from sklearn.preprocessing import LabelEncoder

encoded_df = car_df.copy()
label_encoders = {}

for col in car_df.columns:
  le = LabelEncoder()
  encoded_df[col] = le.fit_transform(car_df[col])
  label_encoders[col] = le

In [461]:
from sklearn.model_selection import train_test_split

X = encoded_df.drop(columns=[class_name])
y = encoded_df[class_name]

X_train, X_not_train, y_train, y_not_train = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_not_train, y_not_train, test_size=0.5, random_state=42, stratify=y_not_train)

In [462]:
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (1209, 6)
X_val shape: (259, 6)
X_test shape: (260, 6)


## Multilayer Perceptron

In [463]:
class DenseLayer:

  def __init__(self, neurons, activation):
    self.neurons = neurons
    self.activation = activation

    self.weights_ = None
    self.biases_ = None

  def forward(self, inputs):
    return self.activation(np.dot(inputs, self.weights_) + self.biases_)

class MultilayerPerceptron:

  def __init__(self, input_size, layers):
    self.input_size = input_size
    self.layers = layers

    n_weights = input_size
    for layer in self.layers:
      layer.weights_ = np.random.normal(0, 1, size=(n_weights, layer.neurons))
      layer.biases_ = np.zeros(layer.neurons)
      n_weights = layer.neurons

  def forward(self, inputs):
    x = inputs
    for layer in self.layers:
      x = layer.forward(x)
    return x

In [464]:
def relu(a):
  return np.maximum(0, a)

def linear(a):
  return a

## Genetic Operators

In [465]:
class OnePointCrossover:

  def __init__(self):
    pass

  def crossover(self, parent1, parent2):
    cut_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:cut_point], parent2[cut_point:]))
    child2 = np.concatenate((parent2[:cut_point], parent1[cut_point:]))
    return child1, child2

In [466]:
class GaussainPointMutation:

  def __init__(self, mutation_strength):
    self.mutation_strength = mutation_strength

  def mutate(self, chromosome):
    mutated_chromosome = chromosome.copy()
    mutated_index = np.random.randint(len(chromosome))
    mutated_chromosome[mutated_index] += np.random.normal(0, self.mutation_strength)
    return mutated_chromosome

In [467]:
class RouletteWheelSelection:
  def __init__(self):
    self.population = None
    self.selection_probabilities = None

  def fit(self, population, fitnesses):
    self.population = population

    total_fitness = np.sum(fitnesses)
    self.selection_probabilities = fitnesses / total_fitness

  def select(self, n=1):
    selected_indices = np.random.choice(
        range(len(self.population)),
        size=n,
        p=self.selection_probabilities
    )
    return self.population[selected_indices]

In [468]:
import numpy as np

class MLPChromosomeAdapter:

  def __init__(self, mlp):
    self.mlp = mlp
    self.weight_shapes = []
    self.bias_shapes = []

    n_inputs = mlp.input_size
    for layer in mlp.layers:
      self.weight_shapes.append((n_inputs, layer.neurons))
      self.bias_shapes.append((layer.neurons,))
      n_inputs = layer.neurons

    self.weight_shapes = np.array(self.weight_shapes)
    self.bias_shapes = np.array(self.bias_shapes)

  def weights_to_chromosome(self):
    chromosome = []
    for layer in self.mlp.layers:
      chromosome.append(layer.weights_.flatten())
      chromosome.append(layer.biases_.flatten())
    return np.concatenate(chromosome)

  def chromosome_to_weights(self, chromosome):
    chromosome_index = 0
    for i, layer in enumerate(self.mlp.layers):
      total_weights = np.prod(self.weight_shapes[i])
      layer.weights_ = chromosome[chromosome_index:(chromosome_index + total_weights)].reshape(self.weight_shapes[i])
      chromosome_index += total_weights

      total_biases = np.prod(self.bias_shapes[i])
      layer.biases_ = chromosome[chromosome_index:(chromosome_index + total_biases)].reshape(self.bias_shapes[i])
      chromosome_index += total_biases

In [469]:
def softmax(a):
  a_max = np.max(a, axis=-1, keepdims=True)
  exps = np.exp(a - a_max)
  return exps / np.sum(exps, axis=-1, keepdims=True)

class ClassificationFitness:
  def __init__(self, nn):
    self.nn = nn

  def evaluate(self, X, y):
    logits = self.nn.forward(X)
    probs = softmax(logits)
    y_pred = np.argmax(probs, axis=1)
    accuracy = np.mean(y_pred == y)

    return accuracy

In [470]:
class GATerminator:

  def __init__(self, patience, max_generations, min_delta=1e-6):
    self.patience = patience
    self.max_generations = max_generations
    self.min_delta = min_delta
    self.best_fitness = None
    self.generations_without_improvement = 0

  def record(self, current_best_fitness):
    if self.best_fitness is None or (current_best_fitness - self.best_fitness) > self.min_delta:
      self.best_fitness = current_best_fitness
      self.generations_without_improvement = 0
    else:
      self.generations_without_improvement += 1

  def should_terminate(self, current_best_fitness, current_generation):
    self.record(current_best_fitness)
    if current_generation >= self.max_generations:
      return True
    if self.generations_without_improvement >= self.patience:
      return True
    return False

In [471]:
import numpy as np

class MLPGeneticAlgorithm:

  def __init__(self, mlp, mutation_strength, max_generations=100, patience=5, min_delta=1e-6):
    self.mlp = mlp
    self.mutation = GaussainPointMutation(mutation_strength)
    self.selection = RouletteWheelSelection()
    self.crossover = OnePointCrossover()
    self.terminator = GATerminator(patience, max_generations, min_delta)

    self.chromosome_adapter = MLPChromosomeAdapter(self.mlp)
    self.fitness = ClassificationFitness(self.mlp)

  def fit(self, X, y, population_size=100, mutation_rate=0.5):
    current_generation = 0

    base_chromosome = self.chromosome_adapter.weights_to_chromosome()
    chromosome_length = base_chromosome.shape[0]
    population = np.array([base_chromosome.copy() + np.random.normal(0, 1, chromosome_length)
                           for _ in range(population_size)])

    fitness_scores = np.array([self.evaluate_chromosome(i, X, y) for i in population])
    best_fitness_score = np.max(fitness_scores)

    while not self.terminator.should_terminate(best_fitness_score, current_generation):
      self.selection.fit(population, fitness_scores)
      new_population = []

      for _ in range(population_size // 2):
        parent1, parent2 = self.selection.select(2)
        child1, child2 = self.crossover.crossover(parent1, parent2)

        if np.random.rand() < mutation_rate:
          child1 = self.mutation.mutate(child1)
        if np.random.rand() < mutation_rate:
          child2 = self.mutation.mutate(child2)

        new_population.append(child1)
        new_population.append(child2)

      population = np.array(new_population)

      fitness_scores = np.array([self.evaluate_chromosome(i, X, y) for i in population])
      best_fitness_score = np.max(fitness_scores)
      current_generation += 1

      print(f"Scores: " + str(fitness_scores))
      print(f"Best fitness score in generation {str(current_generation)}: " + str(best_fitness_score))

    best_index = np.argmax(fitness_scores)
    self.chromosome_adapter.chromosome_to_weights(population[best_index])
    return population, fitness_scores

  def evaluate_chromosome(self, chromosome, X, y):
    self.chromosome_adapter.chromosome_to_weights(chromosome)
    return self.fitness.evaluate(X, y)


In [472]:
mlp = MultilayerPerceptron(6, [
    DenseLayer(128, relu),
    DenseLayer(4, linear)
])

In [473]:
ga = MLPGeneticAlgorithm(mlp, mutation_strength=1, max_generations=250, patience=5)

In [474]:
ga.fit(X_train.values, y_train.values, mutation_rate=0.3)

Scores: [0.1993383  0.05872622 0.10173697 0.46484698 0.05128205 0.13813069
 0.03722084 0.13234078 0.13316791 0.06451613 0.40942928 0.47311828
 0.39867659 0.26385443 0.45657568 0.19602978 0.15715467 0.4681555
 0.18444996 0.26633581 0.07278743 0.29859388 0.15301902 0.38130687
 0.11166253 0.03722084 0.11745244 0.52688172 0.32092639 0.27543424
 0.35897436 0.51447477 0.47146402 0.39536807 0.0942928  0.18114144
 0.48883375 0.0719603  0.07609595 0.28618693 0.07526882 0.41770058
 0.50289495 0.13978495 0.58312655 0.16294458 0.24234905 0.54673284
 0.681555   0.07526882 0.67824648 0.20016543 0.18114144 0.39205955
 0.24565757 0.06286187 0.45492142 0.06203474 0.57568238 0.12489661
 0.42018197 0.45409429 0.19685691 0.5789909  0.31513648 0.56823821
 0.19272126 0.15053763 0.53598015 0.36311001 0.08105873 0.09346567
 0.39950372 0.17287014 0.05872622 0.14557486 0.20347395 0.3407775
 0.38875103 0.33333333 0.67328371 0.42679901 0.41191067 0.07857734
 0.3961952  0.58064516 0.32671629 0.26054591 0.32671629 

(array([[ 1.19623463,  2.01178152, -0.52935545, ...,  0.37964489,
         -0.08731624, -1.58764328],
        [-1.35318022,  1.77870432, -0.2167342 , ...,  0.03766371,
          0.76443695,  0.37603278],
        [ 2.26918125,  0.03394632,  1.17221816, ..., -0.85946544,
          0.60471788,  0.31096382],
        ...,
        [ 0.05609066,  0.78512379, -0.67894188, ..., -0.68624071,
          1.71930917,  0.29929026],
        [ 0.13918774,  1.41192644,  0.72206691, ..., -0.68624071,
          1.71930917,  0.29929026],
        [ 0.05609066,  0.78512379, -0.67894188, ...,  0.37964489,
         -0.08731624, -1.58764328]]),
 array([0.56575682, 0.6592225 , 0.33912324, 0.57485525, 0.16542597,
        0.68651778, 0.6898263 , 0.66418528, 0.69230769, 0.318445  ,
        0.51447477, 0.3490488 , 0.27047146, 0.43093466, 0.5070306 ,
        0.50537634, 0.65508685, 0.10752688, 0.19851117, 0.69478908,
        0.62779156, 0.64185277, 0.02150538, 0.62448304, 0.60711332,
        0.67659222, 0.45409429, 0

In [475]:
ga.fitness.evaluate(X_val.values, y_val.values)

np.float64(0.6988416988416989)

In [476]:
ga.fitness.evaluate(X_test.values, y_test.values)

np.float64(0.7)